<a href="https://colab.research.google.com/github/valentin-popov/mnist_pytorch_tensorflow/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**This notebook shows two examples of neural networks based on Pytorch and Keras. The dataset used is MNIST.**

### **1. Simple Neural Network using Pytorch.**
A simple fully connected feedforward MLP-like network with an input layer (size = 28*28 = 784), an output layer (size = 10 classes) and a hidden layer. Choosing the number of neurons on the hidden layer is a matter of choice between good accuracy and low complexity.

In [48]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
mnist = fetch_openml('mnist_784')

In [49]:
import numpy as np
# print(max(mnist.data[19])) # float64 [0.0 - 255.0] pixel values
# print(type(int(mnist.target[19]))) #str -> int - labels

data = np.array(mnist.data)
target = np.array(mnist.target).astype('int32')
# Scaling pixels values to [0, 1]
x = (data / 255).astype('float32')

# Converting the labels [0, 1, ..., 9] from str to int
y = target.astype(int)

#Splitting into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)


In [52]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from skorch import NeuralNetClassifier

#MLP layers dimensions
input_dim = x.shape[1]
hidden_dim = 50 #change the hidden_dim and notice the accuracy change
output_dim = len(np.unique(y))


class Network1(nn.Module):
    def __init__(self,
                 input_dim = input_dim,
                 hidden_dim = hidden_dim,
                 output_dim = output_dim):
      
        super(Network1, self).__init__()

        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)


    def forward(self, res):
        res = torch.tanh(self.hidden(res))
        res = F.softmax(self.output(res), dim = 0)
        return res
  

net = NeuralNetClassifier(Network1, max_epochs = 30, lr = 0.1,)

In [53]:
net.fit(x_train, y_train);

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.3967       0.8926        3.1541  1.1068
      2        3.0847       0.9065        3.0499  1.1109
      3        3.0042       0.9164        2.9919  1.0843
      4        2.9560       0.9240        2.9550  1.1517
      5        2.9233       0.9284        2.9294  1.1256
      6        2.8994       0.9318        2.9106  1.0757
      7        2.8808       0.9340        2.8961  1.0516
      8        2.8659       0.9361        2.8844  1.0765
      9        2.8535       0.9378        2.8748  1.0711
     10        2.8431       0.9398        2.8667  1.0524
     11        2.8341       0.9409        2.8596  1.0674
     12        2.8262       0.9419        2.8534  1.0483
     13        2.8192       0.9426        2.8479  1.0627
     14        2.8129       0.9429        2.8430  1.1272
     15        2.8072       0.9434        2.8385  1.1023
     16        2.8021       0.9

In [55]:
from sklearn.metrics import accuracy_score
y_pred = net.predict(x_test)
print("Accuracy: {}%".format(round(100 * accuracy_score(y_test, y_pred))))

Accuracy: 96%


### **2. Convolutional Neural Network (CNN) using Tensorflow Keras**
Things are a little more complex in a convolutional neural network (CNN) context, but a convolutional architecture gives a higher accuracy. For faster training enable GPU (Runtime -> Change runtime type).



In [2]:
import numpy as np
import tensorflow
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, Flatten


#Fetching the data split into train and test
#x arrays - images, y arrays - labels(classes)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255).astype('float32')
x_test = (x_test / 255).astype('float32')

# The y arrays are turned into 2D arrays. For every row in the array
# the index of the single non-zero element is the right class.
y_train = tensorflow.keras.utils.to_categorical(y_train)
y_test = tensorflow.keras.utils.to_categorical(y_test)

# format = (height, width, no_channels)
# tuple to keras tensor
input = Input((28, 28, 1))
layer1 = Conv2D(16, kernel_size=(3, 3), activation='relu')(input)
layer2 = Conv2D(32, (3, 3), activation='relu')(layer1)
layer3 = MaxPooling2D(pool_size=(3, 3))(layer2)
layer4 = Dropout(0.5)(layer3)
layer5 = Flatten()(layer4)
layer6 = Dense(100, activation='sigmoid')(layer5)
layer7 = Dense(10, activation='softmax')(layer6)
output = layer7

model = Model([input], output)
model.compile(optimizer=tensorflow.keras.optimizers.Adam(),
			loss=keras.losses.categorical_crossentropy,
			metrics=['accuracy'])

In [5]:
# Train the model
model.fit(x_train, y_train, epochs = 10, batch_size = 1000)

score = model.evaluate(x_test, y_test)
print('Loss =', score[0])
print('Accuracy = {}%'.format(round(100*score[1])))


Epoch 1/10
60/60 [==============================] - 3s 42ms/step - loss: 0.0435 - accuracy: 0.9873
Epoch 2/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0420 - accuracy: 0.9880
Epoch 3/10
60/60 [==============================] - 3s 42ms/step - loss: 0.0403 - accuracy: 0.9883
Epoch 4/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0388 - accuracy: 0.9888
Epoch 5/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0368 - accuracy: 0.9895
Epoch 6/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0360 - accuracy: 0.9898
Epoch 7/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0353 - accuracy: 0.9906
Epoch 8/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0344 - accuracy: 0.9896
Epoch 9/10
60/60 [==============================] - 2s 40ms/step - loss: 0.0332 - accuracy: 0.9905
Epoch 10/10
313/313 [==============================] - 1s 4ms/step - loss: 0.0286 - accuracy: 0.9917
Loss = 0